Contents:

Order and Products merged data
Removed Name columns
Renamed dependant column

# Importing Libraries and Customer Data

In [1]:
# Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# data path
path = r'/Users/Evan/Desktop/Instacart Analysis'

In [3]:
# customer data
df_cust = pd.read_csv(os.path.join(path, '02. Data', 'Original Data', 'customers.csv'))

In [4]:
# data check
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Logic and Consistency Check

In [5]:
# type check
df_cust.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [6]:
# missing value check
df_cust.isnull().sum()

user_id             0
First Name      11259
Surnam              0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [7]:
# dropping name columns as they contain missing values and are irrelevant for this analysis
df_cust = df_cust.drop(columns = ['First Name', 'Surnam'], axis = 1)

In [8]:
# checking for duplicates
df_dup = df_cust[df_cust.duplicated()]

In [9]:
# no duplicates found
df_dup

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


In [10]:
df_cust.rename(columns = {'n_dependants':'dependants'}, inplace = True)

In [11]:
df_cust

,user_id,Gender,STATE,Age,date_joined,dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...
206204,168073,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Female,California,27,4/1/2020,1,married,99799


# Importing Order and Product Data

In [12]:
# Order and Product data
df_opm = pd.read_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'opm_merged_agg.pkl'))

In [13]:
# data check
df_opm.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_day_of_week,...,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_item_spend,spend_flag,med_days_since_prior_order,freq_flag
0,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,prior,28.0,6.0,...,mid-range product,regularly busy,regularly busy,average orders,32.0,regular customer,6.935811,low spender,8.0,frequent customer
1,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,prior,30.0,6.0,...,mid-range product,regularly busy,regularly busy,average orders,32.0,regular customer,6.935811,low spender,8.0,frequent customer
2,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,prior,2.0,0.0,...,mid-range product,busiest day,regularly busy,average orders,5.0,new customer,7.930208,low spender,8.0,frequent customer
3,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,prior,1.0,3.0,...,mid-range product,regularly busy,least busy days,average orders,3.0,new customer,4.972414,low spender,9.0,frequent customer
4,1.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,prior,3.0,4.0,...,mid-range product,least busy,regularly busy,average orders,3.0,new customer,4.972414,low spender,9.0,frequent customer


In [14]:
df_opm.dtypes

product_id                    float64
product_name                   object
aisle_id                      float64
department_id                 float64
prices                        float64
order_id                      float64
user_id                       float64
eval_set                       object
order_number                  float64
order_day_of_week             float64
order_hour_of_day             float64
days_since_prior_order        float64
add_to_cart_order             float64
reordered                     float64
price_range_loc                object
busiest day                    object
busiest days                   object
busiest_period_of_day          object
max_order                     float64
loyalty_flag                   object
avg_item_spend                float64
spend_flag                     object
med_days_since_prior_order    float64
freq_flag                      object
dtype: object

# Merging Data

In [15]:
# converting df_cust user)id to float64 for continuity
df_cust['user_id'] = df_cust['user_id'].astype('float')

In [16]:
# data type check
df_cust.dtypes

user_id        float64
Gender          object
STATE           object
Age              int64
date_joined     object
dependants       int64
fam_status      object
income           int64
dtype: object

In [17]:
# merging data
df_ic = df_opm.merge(df_cust, on = 'user_id', indicator = True)

In [18]:
df_ic.to_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'ord_prod_cust_combined.pkl'))